# 필요한 라이브러리 import

In [1]:
import os
import json
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from implicit.als import AlternatingLeastSquares
from sklearn.decomposition import NMF
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import random
from scipy.stats import skew,kurtosis
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [2]:
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

In [3]:
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

In [4]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

# 태그-id간 동기화를 위한 데이터베이스 연결

In [5]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 연결 config 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

In [6]:
# 데이터 베이스 연결
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

In [7]:
# 커서 생성 후 쿼리문 수행
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select tag_id,tag from tag')
tag_db_list=melon_cursor.fetchall()

# tag 딕셔너리 만들기 

In [8]:
# tag를 id로
tag_to_id = {}

# id를 tag로
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']

# tag 전처리 빈도수로 컷

In [9]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [10]:
# 태그 빈도의 분포를 알아보자
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")
tag_freq.columns=['빈도']
tag_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
빈도,29160.0,16.335082,247.011075,1.0,1.0,1.0,3.0,16465.0


In [12]:
# filter된 태그의 id만 남기기
train_df['tag_ids'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tag_ids
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304]


## 컬럼명 한글화

In [13]:
train_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','태그아이디']

In [14]:
train_df['태그수'] = train_df['태그'].map(len)

In [15]:
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,태그아이디,태그수
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],1


# Train Test Split

In [16]:
# 태그가 10개인거 부터 train에 쓸거임
origin_tags = train_df[train_df['태그수']>9]['태그'].tolist()

train_tags = []
test_tags = []

# 태그는 많아봐야 3개 사분위수 75% -> 3개
for tags in origin_tags:
    tag_3p = len(tags)//3
    train_tag = random.sample(tags, tag_3p)
    test_tag = list(set(tags)-set(train_tag))
    train_tags.append(train_tag)
    test_tags.append(test_tag)

In [17]:
cal_tag_hit_df = pd.DataFrame(columns=['태그원본','예측용태그','검증용태그','svd_예측결과','svd_히트'])
cal_tag_hit_df['태그원본'] = origin_tags

In [18]:
cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[힐링, Pop, 트로피컬하우스]","[트렌드, 일렉, 2017, 드라이브, 기분전환, 팝, 운동]",NaN,NaN
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[댄스, 신나는, 드라이브]","[여름, 여행, 일렉, 기분전환, 매장음악, 일렉트로니카, 취향저격]",NaN,NaN
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[알앤비, 기분전환, 잔잔한]","[새벽, 감성, 인디, 이별, 우울, 사랑, 발라드]",NaN,NaN
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[퇴근길, 라운지, 새벽]","[휴식, 매장, 그루브, 드라이브, 기분전환, 세련된, 펍]",NaN,NaN
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[자장가, 잠, 추위]","[일상, 새벽, 매장, 매장음악, 잔잔한, 카페, 추억]",NaN,NaN


# 희소행렬생성 태그,음악 행렬

In [19]:
song_df.colums=['세부장르','발매일','앨범명','앨범ID','가수ID','노래명','대장르','가수명','노래ID']

In [20]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value

In [21]:
genre_big_dict = {}

for k,v in genre_big.items():
    genre_big_dict[k+'00'] = v

# 태그 임베딩

## 세부장르ID와 CODE간 딕셔너리 생성

In [22]:
genre_detail_id_to_code = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_id_to_code[i] = v

In [23]:
len(genre_detail_id_to_code)

224

In [24]:
genre_detail_code_to_id = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_code_to_id[v] = i

## 대장르ID와 CODE간 딕셔너리 생성

In [25]:
genre_big_id_to_code = {}

for i,v in enumerate(list(genre_big_dict.keys())):
    genre_big_id_to_code[i] = v

In [26]:
genre_big_code_to_id = {}

for i,v in enumerate(list(genre_big_dict.keys())):
    genre_big_code_to_id[v] = i

## 노래ID와 세부장르ID간 딕셔너리 생성

In [27]:
# 노래의 ID를 KEY 세부장르의 ID리스트를 ITEM으로 하는 딕셔너리 생성 

song_genre_detail_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_dtl_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_detail_dict[id].append(genre_detail_code_to_id[code])

In [28]:
# 노래의 ID를 KEY 대\장르의 ID리스트를 ITEM으로 하는 딕셔너리 생성 

song_genre_big_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_big_dict[id].append(genre_big_code_to_id[code])

In [29]:
# 플레이리스트의 모든 노래의 세부장르를 모아서 새로운 컬럼으로 생성

def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_detail_dict[song]
        for id in genre_ids:
            temp.append(id)
    return temp

train_df['세부장르'] = train_df['노래들'].map(fetcher)
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,태그아이디,태그수,세부장르
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],1,"[97, 96, 48, 49, 56, 67, 60, 56, 68, 48, 49, 5..."


In [30]:
# 플레이리스트의 모든 노래의 세부장르를 모아서 새로운 컬럼으로 생성

def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_big_dict[song]
        for id in genre_ids:
            temp.append(id)
    return temp

train_df['대장르'] = train_df['노래들'].map(fetcher)
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,태그아이디,태그수,세부장르,대장르
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],1,"[97, 96, 48, 49, 56, 67, 60, 56, 68, 48, 49, 5...","[13, 8, 9, 9, 8, 9, 8, 9, 8, 9, 9, 8, 12, 13, ..."


## 태그ID와 세부장르ID 간 딕셔너리 생성

In [31]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_detail_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['세부장르'].tolist()):
    for id in ids:
        tag_id_to_genre_detail_ids[id].extend(genres)

## 세부장르의 비율로 데이터 전환

In [32]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_detail_ids.items():
    tag_id_to_genre_detail_ids[k] = dict(Counter(v))

In [33]:
# 딕셔너리의 빈도를 min_max_scaling하고 1을 더함 -> 0이 되는것은 싫다!
for k,value_dict in tag_id_to_genre_detail_ids.items():
    max_val = np.max(list(value_dict.values()))
    min_val = np.min(list(value_dict.values()))
    for key,value in value_dict.items():
        value_dict[key] = np.round((value-min_val)/(max_val-min_val),3)

## 만든 딕셔너리를 이용해서 csr_matrix 생성

In [34]:
# 행,열,데이터 list를 생성
row = []
col = []
dat = []

for k,v in tag_id_to_genre_detail_ids.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

In [35]:
# 29160의 태그의 가짓수, 224는 세부장르의 가짓수
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 224))

In [36]:
# 자료형 변경
A = A.astype(float)

In [37]:
# csr_matrix를 저장
spr.save_npz('./data/tag_genred_csr.npz',A)

In [38]:
tags_frequency['기분전환']

16465

# Vectorization with SVD

## 태그-장르 행렬의 SVD 행렬분해 수행 

In [39]:
# svd행렬분해 100개의 sigular vector사용
u,s,vt = spr.linalg.svds(A,k=100)
svd_vectors = u
data_len,dim = svd_vectors.shape

## cosine유사도를 이용해서 KNN 모델 생성

In [40]:
# 100차원으로 index 생성 및 초기화
svd_p = hnswlib.Index(space='cosine', dim=dim)  
svd_p.init_index(max_elements=data_len, ef_construction=100, M=100)

# 짐재행렬 추가
svd_p.add_items(svd_vectors,np.arange(data_len))

In [41]:
#svd knn모델 저장
with open('./data/svd_knn_model.pickle', 'wb') as f:
    pickle.dump(svd_p,f)

In [42]:
# svd tag의 벡터들을 저장
with open('./data/svd_tag_vectors.pickle', 'wb') as f:
    pickle.dump(svd_vectors,f)

In [43]:
# svd_knn_model로 태그를 예측하여 예측된 결과의 list를 저장하는 함수

def tag_svd(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+svd_vectors[id]
    
    labels, distances = svd_p.knn_query(vectors/len(target_ids), k = 10+len(x))

    ids = [label for label in labels[0]]
    
    return list(set([id_to_tag[tag] for tag in ids])-set(x))

In [44]:
# 예측수행
cal_tag_hit_df['svd_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_svd)
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[힐링, Pop, 트로피컬하우스]","[트렌드, 일렉, 2017, 드라이브, 기분전환, 팝, 운동]","[EDM, 청량감, EDMFloor, 일렉트로닉팝, Electronic, elect...",NaN
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[댄스, 신나는, 드라이브]","[여름, 여행, 일렉, 기분전환, 매장음악, 일렉트로니카, 취향저격]","[더위, 댄스음악, 바다, 기분전환, 휴가, 댄스뮤직, 여름노래, 바캉스, 여름휴가...",NaN
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[알앤비, 기분전환, 잔잔한]","[새벽, 감성, 인디, 이별, 우울, 사랑, 발라드]","[알엔비, 분위기있는, 슬로우잼, PBRnB, 귀르가즘, Sexy, RnB, 끈적한...",NaN
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[퇴근길, 라운지, 새벽]","[휴식, 매장, 그루브, 드라이브, 기분전환, 세련된, 펍]","[쓸쓸한, 감성, 쓸쓸함, 외로움, 우울, 공감, 혼자, 밤, 매장음악, 카페, 쓸...",NaN
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[자장가, 잠, 추위]","[일상, 새벽, 매장, 매장음악, 잔잔한, 카페, 추억]","[생각, 북카페, 불면증, 피곤, 잠들기전, 침대, 집중, 감성자극, 머리를맑게해주...",NaN


In [45]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
svd_tag = cal_tag_hit_df['svd_예측결과'].tolist()

In [46]:
svd_hit = []

for val,svd in zip(val_tag,svd_tag):
    if len(val) == len(set(val)-set(svd)):
        svd_hit.append(0)
    else:
        svd_hit.append(1)

In [47]:
cal_tag_hit_df['svd_히트'] = svd_hit

In [48]:
cal_tag_hit_df

,태그원본,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[힐링, Pop, 트로피컬하우스]","[트렌드, 일렉, 2017, 드라이브, 기분전환, 팝, 운동]","[EDM, 청량감, EDMFloor, 일렉트로닉팝, Electronic, elect...",0
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[댄스, 신나는, 드라이브]","[여름, 여행, 일렉, 기분전환, 매장음악, 일렉트로니카, 취향저격]","[더위, 댄스음악, 바다, 기분전환, 휴가, 댄스뮤직, 여름노래, 바캉스, 여름휴가...",1
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[알앤비, 기분전환, 잔잔한]","[새벽, 감성, 인디, 이별, 우울, 사랑, 발라드]","[알엔비, 분위기있는, 슬로우잼, PBRnB, 귀르가즘, Sexy, RnB, 끈적한...",0
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[퇴근길, 라운지, 새벽]","[휴식, 매장, 그루브, 드라이브, 기분전환, 세련된, 펍]","[쓸쓸한, 감성, 쓸쓸함, 외로움, 우울, 공감, 혼자, 밤, 매장음악, 카페, 쓸...",0
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[자장가, 잠, 추위]","[일상, 새벽, 매장, 매장음악, 잔잔한, 카페, 추억]","[생각, 북카페, 불면증, 피곤, 잠들기전, 침대, 집중, 감성자극, 머리를맑게해주...",0
...,...,...,...,...,...
6896,"[집중, 따스한, 노동요, 아침, 연주곡, 오후, 피아노, 찬바람, 공부, 겨울감성]","[아침, 연주곡, 피아노]","[겨울감성, 찬바람, 집중, 노동요, 따스한, 오후, 공부]","[헤드폰, 시타르, 기리보이의노래는길이보인다, 비정한세상, 부전자전, 메탈리카, 달...",0
6897,"[휴식, 매장음악, 봄, 잔잔한, 취향저격, 커피, 센치, 위로, 감성, 기분전환]","[기분전환, 감성, 봄]","[커피, 휴식, 센치, 매장음악, 잔잔한, 위로, 취향저격]","[산책, 퇴근길, 가사, 노래, 사랑, 공감, 혼자, 우울, 카페, 쓸쓸]",0
6898,"[출근길, 노동요, 연주곡, 카페, 매장, 모닝콜, 휴식, 피아노, 쌀쌀한, 아침]","[출근길, 연주곡, 카페]","[휴식, 매장, 쌀쌀한, 아침, 모닝콜, 노동요, 피아노]","[생각, 북카페, 월요병, 포근한, 불면증, 피곤, 잠들기전, 집중, 머리를맑게해주...",0
6899,"[감성, 잔잔한, 취향저격, 팝송음악, Pop, 휴식, 팝송, 기분전환, 힐링, 신나는]","[잔잔한, 휴식, 감성]","[신나는, 팝송음악, 기분전환, Pop, 팝송, 취향저격, 힐링]","[새벽, 산책, 비오는날, 가을, 밤, 카페, 쓸쓸, 그리움, 겨울, 힐링]",1


In [49]:
sum(svd_hit)/len(svd_hit)

0.6044051586726561

In [51]:
cal_tag_hit_df.head(30)

,태그원본,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[힐링, Pop, 트로피컬하우스]","[트렌드, 일렉, 2017, 드라이브, 기분전환, 팝, 운동]","[EDM, 청량감, EDMFloor, 일렉트로닉팝, Electronic, elect...",0
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[댄스, 신나는, 드라이브]","[여름, 여행, 일렉, 기분전환, 매장음악, 일렉트로니카, 취향저격]","[더위, 댄스음악, 바다, 기분전환, 휴가, 댄스뮤직, 여름노래, 바캉스, 여름휴가...",1
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[알앤비, 기분전환, 잔잔한]","[새벽, 감성, 인디, 이별, 우울, 사랑, 발라드]","[알엔비, 분위기있는, 슬로우잼, PBRnB, 귀르가즘, Sexy, RnB, 끈적한...",0
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[퇴근길, 라운지, 새벽]","[휴식, 매장, 그루브, 드라이브, 기분전환, 세련된, 펍]","[쓸쓸한, 감성, 쓸쓸함, 외로움, 우울, 공감, 혼자, 밤, 매장음악, 카페, 쓸...",0
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[자장가, 잠, 추위]","[일상, 새벽, 매장, 매장음악, 잔잔한, 카페, 추억]","[생각, 북카페, 불면증, 피곤, 잠들기전, 침대, 집중, 감성자극, 머리를맑게해주...",0
5,"[파워워킹, 흥겨운, 출근길, 알앤비, 분위기, 드라이브, 그루브, 맥주, 비트, ...","[비트, 파워워킹, 알앤비]","[출근길, 분위기, 흥겨운, 맥주, 그루브, 퇴근길, 드라이브]","[알엔비, 슬로우잼, groove, PBRnB, 그루브, 귀르가즘, Sexy, Rn...",1
6,"[연말, 저녁, 밤, 캐럴, 눈, 오후, 잠들기전, Lofi, 재즈힙합, 자장가]","[Lofi, 잠들기전, 연말]","[캐럴, 저녁, 밤, 오후, 재즈힙합, 자장가, 눈]","[주말, 인스트루멘탈, Chillhop, jazzhiphop, 로파이, 집중, 나만...",1
7,"[비오는날, 감성, 우울한, Pop, 그루브, 위로, 기분전환, 팝송, 그냥, 팝]","[그루브, 비오는날, 그냥]","[팝송, 기분전환, 팝, Pop, 감성, 우울한, 위로]","[트렌디, 노래추천, 꿀성대, 장르불문, 맥주, 밤에, 내취향, 음색깡패, 힙한, ...",0
8,"[휴식, 밤, 잔잔한, 새벽에, 새벽, Pop, 몽환한, 감성, 팝, Chill]","[잔잔한, Chill, 새벽에]","[휴식, 새벽, 몽환한, 팝, Pop, 밤, 감성]","[칠링, groove, 분위기, Groovy, mood, 멜랑꼴리, 밤에, Lofi...",0
9,"[힙합, 여행, 휴식, 알앤비, 오후, 드라이브, 그루브, 힐링, 기분전환, 트렌디]","[기분전환, 그루브, 힐링]","[여행, 휴식, 트렌디, 알앤비, 드라이브, 힙합, 오후]","[여행, 혼술, 휴식, 트렌디, 리드미컬, 매장음악, 힙스터, 힙한, 음악, 느낌있는]",1


## 과연 한 태그의 장르 점유율은 어떨까?

In [ ]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_detail_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['세부장르'].tolist()):
    for id in ids:
        tag_id_to_genre_detail_ids[id].extend(genres)

In [ ]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_detail_ids.items():
    tag_id_to_genre_detail_ids[k] = dict(Counter(v))

In [ ]:
tag_genre_sorted = {}
for k,v in tag_id_to_genre_detail_ids.items():
    tag_genre_sorted[k] = sorted(tag_id_to_genre_detail_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
genre_counts = []

for k,v in tag_genre_sorted.items():
    genre_counts.append([count for id,count in v])

In [ ]:
genre_skew = list(map(lambda x : skew(x),genre_counts))

In [ ]:
genre_kurtosis = list(map(lambda x : kurtosis(x),genre_counts))

In [ ]:
skew(genre_counts[2])

In [ ]:
np.median(genre_counts[2])

In [ ]:
max(genre_kurtosis), min(genre_kurtosis)

In [ ]:
np.median(genre_kurtosis)

In [ ]:
genre_kurtosis.index(min(genre_kurtosis))

In [ ]:
genre_kurtosis

In [ ]:
len(genre_skew)

In [ ]:
#sns.kdeplot([(x-np.mean(genre_counts[0]))/np.std(genre_counts[0]) for x in genre_counts[0]],shade=True)
#sns.kdeplot(np.random.normal(),shade=True)
#np.mean(genre_counts[2])

#ax = sns.kdeplot(genre_counts[2],shade=False,color='crimson')
ax = sns.distplot(genre_counts[1099],color='crimson')
kdeline = ax.lines[0]
mean = np.mean(genre_counts[1099])
xs = kdeline.get_xdata()
ys = kdeline.get_ydata()
height = np.interp(mean,xs,ys)
ax.vlines(mean,0,height,color='crimson',ls=':')
ax.fill_between(xs,0,ys,facecolor='crimson',alpha=0.2)
plt.show()
print(f' 평균 : {mean}  왜도: {skew(genre_counts[1099])} 첨도: {kurtosis(genre_counts[1099])}')

In [ ]:
id_to_tag[1099]

In [ ]:
tag_id_to_genre_big_ids[2]

In [ ]:
genre_counts_pie = []

genre_counts_pie.extend(genre_counts[2])
genre_counts_pie.append(np.sum(genre_counts[2][10:]))

plt.pie(genre_counts_pie)

plt.show()

# 과연 한 태그의 대장르의 분포가 치우쳐져 있을까?

In [ ]:
# {tag_Id:genre_big_ids}

tag_id_to_genre_big_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['대장르'].tolist()):
    for id in ids:
        tag_id_to_genre_big_ids[id].extend(genres)

In [ ]:
# 대장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_big_ids.items():
    tag_id_to_genre_big_ids[k] = dict(Counter(v))

In [ ]:
tag_big_genre_sorted = {}
for k,v in tag_id_to_genre_big_ids.items():
    tag_big_genre_sorted[k] = sorted(tag_id_to_genre_big_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
big_genre_counts = []

for k,v in tag_big_genre_sorted.items():
    big_genre_counts.append([count for id,count in v])

In [ ]:
big_genre_counts_pie = []

big_genre_counts_pie.extend(big_genre_counts[2])
#big_genre_counts_pie.append(np.sum(big_genre_counts[2][5:]))

plt.pie(big_genre_counts_pie)

plt.show()

# 과연 플레이리스트안의 세부장르의 분포는?

In [ ]:
train_df['새플리아이디'] = train_df.index
train_df.head(1)

In [ ]:
p_id_to_genre_detail_ids = defaultdict(list)

for id,genres in zip(train_df['새플리아이디'].tolist(),train_df['세부장르'].tolist()):
    p_id_to_genre_detail_ids[id].extend(genres)

In [ ]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in p_id_to_genre_detail_ids.items():
    p_id_to_genre_detail_ids[k] = dict(Counter(v))

In [ ]:
p_genre_sorted = {}
for k,v in p_id_to_genre_detail_ids.items():
    p_genre_sorted[k] = sorted(p_id_to_genre_detail_ids[k].items(),key=lambda x : x[1],reverse=True)

In [ ]:
p_genre_counts = []

for k,v in p_genre_sorted.items():
    p_genre_counts.append([count for id,count in v])

In [ ]:
p_genre_skew = list(map(lambda x : skew(x),p_genre_counts))

In [ ]:
max(p_genre_skew), min(p_genre_skew)

In [ ]:
p_genre_kurtosis = list(map(lambda x : kurtosis(x),p_genre_counts))

In [ ]:
max(p_genre_kurtosis), min(p_genre_kurtosis)